In [3]:
import torch 
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import os


In [7]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)
    
class Corpus(object):
    def __init__(self, path='./data'):
        self.dictionary = Dictionary()
        self.train = os.path.join(path, 'train.txt')
        self.test = os.path.join(path, 'test.txt')

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)
    
# RNN Based Language Model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.init_weights()
        
    def init_weights(self):
        self.embed.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)
        self.linear.weight.data.uniform_(-0.1, 0.1)
        
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x) 
        
        # Forward propagate RNN  
        out, h = self.lstm(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.contiguous().view(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time step
        out = self.linear(out)  
        return out, h


In [5]:
# Hyper Parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000   # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [6]:
# Load Penn Treebank Dataset
train_path = './data/train.txt'
sample_path = './sample.txt'
corpus = Corpus()
ids = corpus.get_data(train_path, batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length


In [8]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Truncated Backpropagation 
def detach(states):
    return [Variable(state.data) for state in states] 


In [10]:
# Training
for epoch in range(num_epochs):
    # Initial hidden and memory states
    states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)),
              Variable(torch.zeros(num_layers, batch_size, hidden_size)))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get batch inputs and targets
        inputs = Variable(ids[:, i:i+seq_length])
        targets = Variable(ids[:, (i+1):(i+1)+seq_length].contiguous())
        
        # Forward + Backward + Optimize
        model.zero_grad()
        states = detach(states)
        outputs, states = model(inputs, states) 
        loss = criterion(outputs, targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [%d/%d], Step[%d/%d], Loss: %.3f, Perplexity: %5.2f' %
                   (epoch+1, num_epochs, step, num_batches, loss.data[0], np.exp(loss.data[0])))


Epoch [1/5], Step[0/1549], Loss: 9.212, Perplexity: 10012.28
Epoch [1/5], Step[100/1549], Loss: 6.026, Perplexity: 414.23
Epoch [1/5], Step[200/1549], Loss: 5.932, Perplexity: 376.74
Epoch [1/5], Step[300/1549], Loss: 5.804, Perplexity: 331.55
Epoch [1/5], Step[400/1549], Loss: 5.696, Perplexity: 297.77
Epoch [1/5], Step[500/1549], Loss: 5.142, Perplexity: 171.08
Epoch [1/5], Step[600/1549], Loss: 5.206, Perplexity: 182.31
Epoch [1/5], Step[700/1549], Loss: 5.402, Perplexity: 221.81
Epoch [1/5], Step[800/1549], Loss: 5.212, Perplexity: 183.39
Epoch [1/5], Step[900/1549], Loss: 5.108, Perplexity: 165.26
Epoch [1/5], Step[1000/1549], Loss: 5.165, Perplexity: 174.96
Epoch [1/5], Step[1100/1549], Loss: 5.335, Perplexity: 207.46
Epoch [1/5], Step[1200/1549], Loss: 5.166, Perplexity: 175.21
Epoch [1/5], Step[1300/1549], Loss: 5.094, Perplexity: 163.09
Epoch [1/5], Step[1400/1549], Loss: 4.825, Perplexity: 124.56
Epoch [1/5], Step[1500/1549], Loss: 5.181, Perplexity: 177.91
Epoch [2/5], Step[

In [13]:
# Sampling
with open(sample_path, 'w') as f:
    # Set intial hidden ane memory states
    state = (Variable(torch.zeros(num_layers, 1, hidden_size)),
         Variable(torch.zeros(num_layers, 1, hidden_size)))

    # Select one word id randomly
    prob = torch.ones(vocab_size)
    input = Variable(torch.multinomial(prob, num_samples=1).unsqueeze(1),
                     volatile=True)
    s = ""
    for i in range(num_samples):
        # Forward propagate rnn 
        output, state = model(input, state)
        
        # Sample a word id
        prob = output.squeeze().data.exp()
        word_id = torch.multinomial(prob, 1)[0]
        
        # Feed sampled word id to next time step
        input.data.fill_(word_id)
        
        # File write
        word = corpus.dictionary.idx2word[word_id]
        word = '\n' if word == '<eos>' else word + ' '
        s += word
    print (s)


version <unk> who jones chairman dennis <unk> r. utah in a position in N after he was a <unk> 
most former chairmen had <unk> been called <unk> for N most years ago at the agency and its <unk> texas energy operation 
so far crunch can close to de new job and also <unk> their own wives 
it can even maintain its music above it 
<unk> is the crown prince and <unk> operation supermarket & <unk> 
common <unk> prosecutors featured a $ <unk> a <unk> <unk> <unk> to a design when a treatment made a stronger debate toward <unk> <unk> in N 
for everyone frightened it was to leave it clear even 
i would n't find who readers are going into all scenes going off or even never yet to be abc or <unk> to baltimore 
eventually they <unk> he says 
though it continues to convey that schedules into the basic system still <unk> protein to post which were one of many <unk> students expectations and federal <unk> salaries after the third game after the time 
the league is to draw someone again when you will bu